# Optimization with MPOpt

This is a demo which will show you the whole process of building, testing and analyzing your algorithm with `mpopt`.

In [2]:
import numpy as np

## 1. Objective Function

`mpopt` use 'ObjFunction' as an abstraction of objective function. It contains the callable function and extra information needed for optimizer. 

For each run of optimization, an 'Evaluator' instance should be created and passed to the optimization algorithm. It holds all the information about the task and will automatically update states when some solutions are evaluated. 

Compared with the 'ask-tell' interface, this framework provides much more flexibility for optimization algorithm.

You can get a standard evaluator from our provided benchmarks with following codes:

In [3]:
from mpopt.benchmarks.benchmark import Benchmark
# get benchmark CEC20 with 10 dim
benchmark = Benchmark('CEC20', 10)
# get an evaluator for the first function in benchmark
evaluator = benchmark.generate(0)

Or you are free to build your own objective functions and evaluators. For example:

In [4]:
from mpopt.tools.objective import ObjFunction, Evaluator

# an callable objective function
def my_obj(x):
    return np.sum(x**2)

# create objective function
# information about my_obj is needed
obj = ObjFunction(my_obj, dim=2, lb=-1, ub=1,)

# create evaluator
# information about the optimization is need
evaluator = Evaluator(obj, max_eval=100)

Now the evaluator is ready for an optimization. For example, this is how random-search be implemented in our framework. 

In [5]:
# alias
e = evaluator
dim = e.obj.dim
lb = e.obj.lb
ub = e.obj.ub

# algorithm begin
while not e.terminate():
    rand_sample = np.random.uniform(lb, ub, (1, dim))
    e(rand_sample)
print("Solution: {}. Val: {:.3e}".format(e.best_x, e.best_y))

Solution: [ 0.02720946 -0.05039847]. Val: 3.280e-03


## 2. Population

Using the evaluator, we can directly start optimization as above. However, here we focus on optimization algorithm based on population(s). 

`mpopt` provide a base class `BasePop` for general population and a base class  `BaseFirework` for firework in `mpopt.population.base`. 

You are supposed to inherit from those class, and re-write methods you changed. New population definition should be stored in 'mpopt/population' directory.

For exmaple, here we build a firework population with dynamic amplitude (which is the same as [BBFWA]):

In [6]:
from mpopt.population.base import BaseFirework

class DynFirework(BaseFirework):
    # re-write __init__
    def __init__(self, idv, val, amp, num_spk, **kwargs):
        super().__init__(idv, val, **kwargs)

        self.amp = amp
        self.num_spk = num_spk

        # set default dynamic ratio if not given
        if 'dr_amp' not in self.__dict__:
            self.dr_amp = 1.2
        if 'dr_red' not in self.__dict__: 
            self.dr_red = 0.9

    # re-write update for dynamic ratios
    def update(self):
        if self.val - self.new_val > 1e-5:
            # improved
            self.amp *= self.dr_amp
        else:
            self.amp *= self.dr_red
        
        # update firework
        self.idv = self.new_idv
        self.val = self.new_val

Since we have default setting for other parts of the firework population, a `DynFirework` instance is ready to evolve with a evaluator once it is created. 

In [7]:
# set environment
e = Evaluator(obj, max_eval=100)
dim = e.obj.dim
lb = e.obj.lb
ub = e.obj.ub

# init pop
idv = np.random.uniform(lb, ub, (dim,))
val = e(idv[np.newaxis,:])[0]

firework = DynFirework(idv, val, ub-lb, 5, lb=lb, ub=ub)
while not e.terminate():
    firework.evolve(e)
print("Solution: {}. Val: {:.3e}".format(e.best_x, e.best_y))

Solution: [0.05369561 0.00543562]. Val: 2.913e-03


## optimization

The cell above is already a runable optimization algorithm based on a single population. However, there are still two important features for an algorithm interface.



### 1. Unified Interface

For the convenience to apply or analysis our algorithm, it is better to unify their interface. We require each algorithm be abstracted as a class and provide following methods:

- `__init__`: Define all the parameters and states (include populations) here for reading.

- `default_params(self, benchmark=None)`: Get default parameters.

- `set_params(self, params)`: Set parameters. 

- `init(self, e)`: Init states of the algorithm.

- `optimize(self, e)`: Run optimization and return the optimal value.



### 2. Collaborate Multiple Populations

With `mpopt`, we hope to easily implement optimizations with multiple populations. Now the evolution method for each population can be pre-defined, we only need to consider the collaboration during each iteration.

In order to maximize the flexibility of the algorithms, we leave all the states and parameters of population opened. So in each itaration, you can just let each population evolve and then do whatever you want.

### Example of LoTFWA (without guided mutation)

We implement a LoTFWA without guided mutation algorithm as an example:

In [8]:
from mpopt.algorithms.base import BaseAlg

class DemoFWA(BaseAlg):

    def __init__(self, obj):
        # populations
        self.fireworks = None
        # params
        self.fw_size = None
        self.init_amp = None
        self.num_spk = None
        # load default
        self.set_params(self.default_params(obj))
    
    def default_params(self, obj):
        params = {}
        params['fw_size'] = 4
        params['init_amp'] = obj.ub - obj.lb
        params['num_spk'] = 5
        return params

    def init(self, e):
        init_pop = np.random.uniform(e.obj.lb, e.obj.ub, (self.fw_size, e.obj.dim))
        init_fit = e(init_pop)
        self.fireworks = [DynFirework(init_pop[i,:], 
                                      init_fit[i], 
                                      self.init_amp, 
                                      self.num_spk,
                                      lb=e.obj.lb,
                                      ub=e.obj.ub,) for i in range(self.fw_size)]
    
    def optimize(self, e):
        self.init(e)
        while not e.terminate():
            # evolve
            for idx in range(self.fw_size):
                self.fireworks[idx].explode()
                self.fireworks[idx].eval(e)
                self.fireworks[idx].select()
            
            # collaborate
            restart = [False] * self.fw_size
            for idx in range(self.fw_size):
                # alias
                fw = self.fireworks[idx]
                # improved
                if fw.val - fw.new_val > 1e-5:
                    rest_iter = (e.max_eval - e.num_eval) / (self.fw_size * self.num_spk)
                    if (fw.val - fw.new_val) * rest_iter > (fw.val - e.cur_y):
                        # restart fw
                        restart[idx] = True
                        restart_pop = np.random.uniform(e.obj.lb, e.obj.ub, (1, e.obj.dim))
                        restart_fit = e(restart_pop) 
                        self.fireworks[idx] = DynFirework(restart_pop[0,:],
                                                          restart_fit[0],
                                                          self.init_amp,
                                                          self.num_spk,
                                                          lb=e.obj.lb,
                                                          ub=e.obj.ub,)
            
            # update populations
            for idx in range(self.fw_size):
                if not restart[idx]:
                    self.fireworks[idx].update()
        
        return e.best_y
    
# optimize
e = Evaluator(obj, max_eval=1000)
alg = DemoFWA(obj)
alg.optimize(e)
print("Solution: {}. Val: {:.3e}".format(e.best_x, e.best_y))

Solution: [-0.03927546 -0.00498935]. Val: 1.567e-03


## Benchmarking

We provide several standard benchmarks to test your algorithm in `mpopt.benchmarks`. As we shown before, `benchmark` instance can generate evaluators which is set according to the requirements of the benchmark.

We also provide a standard testing script for benchmarking in `mpopt/../runs/benchmark_opt.py` which including evaluators generation, timing, multiprocessing, and result recording. You can run a test of LoTFWA on CEC20 by typing following code:

In [9]:
!python ../runs/benchmark_opt.py -b CEC20 -d 10 -a LoTFWA -r 1 -n test

Prob.1   , res:7.2904e+05,	 time:13.904
Prob.2   , res:1.8893e+03,	 time:15.603
Prob.3   , res:7.2601e+02,	 time:14.656
Prob.4   , res:1.9032e+03,	 time:14.404
Prob.5   , res:7.1914e+03,	 time:15.030
Prob.6   , res:1.6102e+03,	 time:15.128
Prob.7   , res:4.4231e+03,	 time:15.182
Prob.8   , res:2.3098e+03,	 time:19.365
Prob.9   , res:2.7394e+03,	 time:19.831
Prob.10  , res:2.8983e+03,	 time:19.960


## Results Comparing

We provide useful comparing script for our formatted result records in `mpopt.tools.result.py`. (The printed results is formatted in terminal.)

Providing two result paths, the script will conduct a statistical comparation:

In [10]:
!python ../mpopt/tools/result.py ../logs/CEC20_10D/AGSK.json ../logs/CEC20_10D/LoTFWA.json --b CEC20 -d 10

Comparing on CEC20: alg1: AGSK, alg2: LoTFWA__test
Win: 0, Lose: 10
+-----+-----------+-----------+-----------+-----------+---------+-----+
| idx | alg1.mean |  alg1.std | alg2.mean |  alg2.std | P-value | Sig |
+-----+-----------+-----------+-----------+-----------+---------+-----+
|  1  | 0.000e+00 | 0.000e+00 | 8.574e+05 | 3.675e+05 |   0.00  |  +  |
|  2  | 2.845e+01 | 3.152e+01 | 5.925e+02 | 2.662e+02 |   0.00  |  +  |
|  3  | 9.925e+00 | 4.186e+00 | 3.703e+01 | 6.706e+00 |   0.00  |  +  |
|  4  | 5.826e-02 | 3.061e-02 | 3.664e+00 | 7.167e-01 |   0.00  |  +  |
|  5  | 3.176e-01 | 3.008e-01 | 2.402e+04 | 2.837e+04 |   0.00  |  +  |
|  6  | 1.547e-01 | 1.154e-01 | 8.700e+01 | 6.906e+01 |   0.00  |  +  |
|  7  | 1.537e-03 | 1.682e-03 | 2.348e+03 | 1.083e+03 |   0.00  |  +  |
|  8  | 1.800e+01 | 2.339e+01 | 7.327e+01 | 3.921e+01 |   0.00  |  +  |
|  9  | 7.629e+01 | 4.214e+01 | 2.406e+02 | 1.203e+02 |   0.00  |  +  |
|  10 | 2.985e+02 | 1.404e+02 | 4.069e+02 | 1.740e+01 |   0.00  |  +

Providing more than two paths or a directory path, the script will conduct a averange ranking comparation:

In [12]:
!python ../mpopt/tools/result.py ../logs/CEC20_10D/ --b CEC20 -d 10

Comparing on CEC20:
+---------+-----------+-----------+-------------------+------------------+------------+-----------+
|   idx   | AGSK.mean |  AGSK.std | LoTFWA__test.mean | LoTFWA__test.std | IMODE.mean | IMODE.std |
+---------+-----------+-----------+-------------------+------------------+------------+-----------+
|    1    | 1.000e+02 | 0.000e+00 |     8.575e+05     |    3.675e+05     | 1.000e+02  | 0.000e+00 |
|    2    | 1.128e+03 | 3.152e+01 |     1.692e+03     |    2.662e+02     | 1.104e+03  | 3.639e+00 |
|    3    | 7.099e+02 | 4.186e+00 |     7.370e+02     |    6.706e+00     | 7.121e+02  | 7.694e-01 |
|    4    | 1.900e+03 | 3.061e-02 |     1.904e+03     |    7.167e-01     | 1.900e+03  | 0.000e+00 |
|    5    | 1.700e+03 | 3.008e-01 |     2.572e+04     |    2.837e+04     | 1.700e+03  | 3.768e-01 |
|    6    | 1.600e+03 | 1.154e-01 |     1.687e+03     |    6.906e+01     | 1.600e+03  | 4.996e-02 |
|    7    | 2.100e+03 | 1.682e-03 |     4.448e+03     |    1.083e+03     | 2.100